In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.cm as cm
from PIL import Image
import gc
import os
import nltk
nltk.download('punkt')
from tqdm import tqdm
import pickle
import skimage.transform
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch.nn.utils.rnn import  pack_padded_sequence

from model import Decoder, EncoderResNet
from utils import  to_variable, tensor2numpy, Vocabulary

In [ ]:
def rescale_feature(img):
    img += abs(img.min())
    img /= img.max()
    return img

In [ ]:
def set_transform():
    transform = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406),
                             (0.229, 0.224, 0.225))])
    
    return transform

In [ ]:
#root_img_dir= "../images/val2014/"
root_img_dirc = "/data/sakka/show_attend_data/val2014/"
label_df = pd.read_csv("../data/label/val.csv")
for i,path in enumerate(label_df["file_name"]):
    label_df.iloc[i]["file_name"] = os.path.join(root_img_dirc, path)

label_df.head()

# vocab, encoder, decoderのpath指定

In [ ]:
vocab_path = "../data/vocab/vocab.pkl"
vocab = pickle.load(open(vocab_path, "rb"))

# Encoder
encoder_model = EncoderResNet()
encoder_model_path = "/data/sakka/show_attend_data/model/encoder.pth"
encoder_model = nn.DataParallel(encoder_model)
encoder_model.load_state_dict(torch.load(encoder_model_path))

# model setting
vis_dim = 2048
vis_num = 196
embed_dim = 512
hidden_dim = 512
vocab_size = 10000
num_layers = 1
dropout_ratio = 0.0

# Decoder
decoder_model = Decoder(vis_dim=vis_dim,
                vis_num=vis_num, 
                embed_dim=embed_dim,
                hidden_dim=hidden_dim, 
                vocab_size=vocab_size, 
                num_layers=num_layers,
                dropout_ratio=dropout_ratio)

decoder_model = nn.DataParallel(decoder_model)

decoder_model_path = "/data/sakka/show_attend_data/model/decoder.pth"
decoder_model.load_state_dict(torch.load(decoder_model_path))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("DEVICE: {}".format(device))

# evaluation mode
encoder_model = encoder_model.to(device)
encoder_model.eval()
decoder_model = decoder_model.to(device)
decoder_model.eval()

In [ ]:
# 
caption_lst = list(label_df["caption"])    
# name_lst = []
# for opinion in opinion_lst:
#     name = ""
#     for char in opinion:
#         name += "{} ".format(char)
#     name_lst.append(name)
 
# initialize
#name_lst = name_lst
img_path_lst = list(label_df["file_name"])
names_lst = []
captions_lst = []
alphas_lst = []
transform = set_transform()
rm_path_cnt = 0

with torch.no_grad():
    for img_path, cap in tqdm(zip(img_path_lst, caption_lst)):
        if os.path.exists(img_path):
            img = Image.open(img_path).convert("RGB")
            img = transform(img)
            img = to_variable(img)
            fea = encoder_model(img.unsqueeze(0))
            fea = fea.view(fea.size(0), 2048, 196).transpose(1,2)

            ids, weights = decoder_model.module.beam_search_captioning(fea, vocab, beam_size=5)
            names_lst.append(cap.lower())
            captions_lst.append(ids)
            alphas_lst.append(weights)
        else:
            rm_path_cnt += 1
            
print("Not exist path: {0}".format(rm_path_cnt))

In [ ]:
print("name: {}".format(names_lst[0]))
print("caption: {}".format(captions_lst[0]))
print("alphas: {}".format(np.array(alphas_lst[0][1]).shape))

In [ ]:
def decode_captions(captions, idx_to_word):
    N, D = captions.shape
    decoded = []
    for idx in range(N):
        words = []
        for wid in range(D):
            word = idx_to_word[captions[idx, wid]]
            if word == '<end>':
                break
            words.append(word)
        decoded.append(words)
        
    return decoded

In [ ]:
def attention_visualization(img_path, caption, alphas):
    image = Image.open(img_path).convert("RGB")
    image = image.resize((224, 224))
    plt.figure(figsize=(12, 12))
    plt.subplot(4,5,1)
    plt.text(0, 1, "<start>" , color='black', backgroundcolor='white', fontsize=8)
    plt.imshow(image)
    plt.axis('off')
    
    words = caption
    total_alp = np.zeros((14, 14))
    for t in range(len(words)):
        if words[t] == "<end>":
            break
        if t > 14:
            break
        plt.subplot(4, 5, t+2)
        plt.text(0, 1, '%s'%(words[t]) , color='black', backgroundcolor='white', fontsize=14)
        plt.imshow(image)

        alp_curr = alphas[t, :].view(14, 14)
        total_alp += alp_curr
        alp_img = skimage.transform.pyramid_expand(alp_curr.numpy(), upscale=16, sigma=20)
        plt.gray()
        plt.imshow(alp_img, alpha=0.8)
        plt.axis('off')
    
    plt.figure()
    plt.imshow(image)
    total_alp = skimage.transform.pyramid_expand(total_alp, upscale=16, sigma=20)
    plt.imshow(total_alp, alpha=0.8)
    plt.show()

In [ ]:
def get_result(alphas_lst, captions_lst, img_path_lst, vocab, idx):
    #alps = torch.cat(tuple(alphas_lst[idx][1:]), 0)
    alps = alphas_lst[idx][1:]
    alps += abs(alps.min())
    alps /= alps.max()
    cap = decode_captions(captions_lst[idx].data.cpu().numpy().reshape(1, -1), vocab.idx2word)[0]
    print("Prediction : {}".format(" ".join(cap)))
    attention_visualization(img_path_lst[idx], cap, alps.data.cpu())

In [ ]:
test_id = 0
print("ID: {}".format(test_id))
print("Answer     : {}".format(names_lst[test_id]))
get_result(alphas_lst, captions_lst, img_path_lst, vocab, test_id)